In [ ]:
# Imports

import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt

plt.style.use("ggplot")

In [ ]:
# Company Choice
ticker_symbol = "TSLA"
ticker = yf.Ticker(ticker_symbol)

# Download financial data
financials = ticker.financials

prices = yf.download(ticker_symbol, period = "2y")

In [ ]:
# Extracting Revenue and Net Income
revenue = financials.loc["Total Revenue"]
net_income = financials.loc["Net Income"]

business_df = pd.DataFrame({
    "Revenue": revenue,
    "Net Income": net_income
})

business_df["Revenue Growth (%)"] = business_df["Revenue"].pct_change()*100
business_df["Profit Margin Growth(%)"] = (business_df["Net Income"] / business_df["Revenue"]).pct_change()*100

business_df

In [ ]:
prices.columns = ["_".join([c for c in col if c])for col in prices.columns]

In [ ]:
# Calculation of daily percentage return

prices["Daily_Return"] = prices["Close_TSLA"].pct_change()

# SMA Crossover

# 20 day Moving Average
prices["SMA20"] = prices["Close_TSLA"].rolling(20).mean()

# Signal 1 - Buy when the price goes above SMA20, sell when price goes below SMA20
prices["Signal"] = (prices["Close_TSLA"] > prices["SMA20"]).astype(int)

# RSI(14)
delta = prices["Close_TSLA"].diff()
gain = delta.where(delta>0,0)
loss = -delta.where(delta<0,0)

average_gain = gain.rolling(window=14, min_periods=14).mean()
average_loss = loss.rolling(window=14, min_periods=14).mean()

rs = average_gain/average_loss
prices["RSI"] = 100 - (100/(1+rs))

prices.tail()

# Signal 2 - Buy when RSI < 30, sell when RSI > 70
prices["RSI_Signal"] = 0
prices.loc[prices["RSI"] < 30, "RSI_Signal"] = 1
prices.loc[prices["RSI"] > 70, "RSI_Signal"] = 0

# MACD

# Calculate EMAs
prices["EMA12"] = prices["Close_TSLA"].ewm(span=12,adjust=False).mean()
prices["EMA26"] = prices["Close_TSLA"].ewm(span=26,adjust=False).mean()

# MACD line
prices["MACD"] = prices["EMA12"]-prices["EMA26"]

# Signal line
prices["MACD_Signal"] = prices["MACD"].ewm(span=9, adjust=False).mean()

# Generation of MACD trading signals - Buy if MACD crosses above signal, sell if MACD crosses below signal
prices["MACD_Signal_Flag"] = 0
prices.loc[prices["MACD"] > prices["MACD_Signal"], "MACD_Signal_Flag"] = 1

In [ ]:
# Returns Calc from Strategy Performances

prices["Strategy_Return"] = prices["Daily_Return"] * prices["Signal"].shift(1).fillna(0)
prices["RSI_Strategy_Return"] = (prices["Daily_Return"] * prices["RSI_Signal"].shift(1).fillna(0))
prices["MACD_Strategy_Return"] = (prices["Daily_Return"] * prices["MACD_Signal_Flag"].shift(1).fillna(0))
buy_hold = (1+prices["Daily_Return"].fillna(0)).cumprod()
strategy = (1+prices["Strategy_Return"].fillna(0)).cumprod()
rsi_strategy = (1+prices["RSI_Strategy_Return"].fillna(0)).cumprod()
macd_strategy = (1+prices["MACD_Strategy_Return"].fillna(0)).cumprod()

results = pd.DataFrame({
    "Buy_and_Hold": buy_hold,
    "Strategy": strategy,
    "RSI_Strategy": rsi_strategy,
    "MACD_Strategy": macd_strategy
})

results.tail()

In [ ]:
# Charting business performance
business_df[["Revenue", "Net Income"]].plot(figsize=(12,5))
plt.title(f"{ticker_symbol} - Revenue vs Net Income")
plt.ylabel("USD ($)")
plt.show()

In [ ]:
# Charting Price vs SMA20

prices[["Close_TSLA", "SMA20"]].plot(figsize=(12,5))
plt.title(f"{ticker_symbol} Price vs 20-day SMA")
plt.ylabel("Price ($)")
plt.show()

In [ ]:
# Chart Strategies vs BH

results.plot(figsize=(12,6))
plt.title(f"{ticker_symbol} Strategy Comparisons")
plt.ylabel("Growth of $1")
plt.show()

In [ ]:
tot_trades = prices["Signal"].diff().abs().sum()

final_buy_hold = buy_hold.iloc[-1]
final_strat = strategy.iloc[-1]
final_rsi_strat = rsi_strategy.iloc[-1]
final_macd_strat = macd_strategy.iloc[-1]

print(f"Total trades by strategy = {int(tot_trades)}")
print(f"Final Buy-Hold growth of $1 = {final_buy_hold:.2f}")
print(f"Final Strategy growth of $1 = {final_strat:.2f}")
print(f"Final RSI Strategy growth of $1 = {final_rsi_strat:.2f}")
print(f"Final MACD Strategy growth of $1 = {final_macd_strat:.2f}")

